In [48]:
# !pip install chromadb

import openai
import chromadb
import pprint
import json

In [49]:
print = pprint.pprint

In [50]:
# Initialize ChromaDB client (can be in-memory or persistent)
# client = chromadb.Client() # In-memory client

client = chromadb.PersistentClient(path="./github-trending-repositories") # Persistent client


In [51]:
collection_name = "repositories"

# Get or create a collection
# If collection exists with wrong dimension, delete and recreate it
try:
    collection = client.get_collection(name=collection_name)
    # Check if dimension matches (text-embedding-3-small produces 384 dimensions)
    # If you need 1536 dimensions, use text-embedding-3-large or specify dimension
    print(f"Collection '{collection_name}' already exists")
except:
    # Create collection - specify dimension if needed
    # Option 1: Use 384 dimensions (text-embedding-3-small default)
    collection = client.create_collection(
        name=collection_name,
        metadata={"description": "GitHub trending repositories"}
    )
    print(f"Created new collection '{collection_name}'")

# If you need to reset the collection (delete and recreate with correct dimension):
# Uncomment the following lines:
# try:
#     client.delete_collection(name=collection_name)
#     print(f"Deleted existing collection '{collection_name}'")
# except:
#     pass
# collection = client.create_collection(
#     name=collection_name,
#     metadata={"description": "GitHub trending repositories"}
# )


"Collection 'repositories' already exists"


In [52]:
import pandas as pd
import glob

# load all csv files from docs folder
one_month_file = glob.glob("../docs/2025/11/2025-11-14.csv")

# read all csv files
df = pd.concat(map(pd.read_csv, one_month_file), ignore_index=True)

In [53]:
print(f"Number of rows: {len(df)}")
print(f"Number of columns: {len(df.columns)}")
print('-'*100)

# print(df.head())
print(f"Showing first row:")
print(df.iloc[0])

'Number of rows: 20'
'Number of columns: 21'
'----------------------------------------------------------------------------------------------------'
'Showing first row:'
full_name                              cryptocj520/crypto-trading-open
stargazers_count                                                  1058
owner.login                                                cryptocj520
owner.avatar_url     https://avatars.githubusercontent.com/u/204736...
description                                        crypto-trading-open
topics                                                              []
html_url             https://github.com/cryptocj520/crypto-trading-...
created_at                                        2025-11-11T12:00:02Z
updated_at                                        2025-11-14T08:42:57Z
pushed_at                                         2025-11-11T12:03:28Z
git_url              git://github.com/cryptocj520/crypto-trading-op...
ssh_url              git@github.com:cryptocj520/cr

In [54]:
import numpy as np

def get_embedding(text, model="text-embedding-3-small", dimensions=1536):
    """Get embedding from OpenAI and convert to list of native Python floats.

    Args:
        text: Text to embed
        dimensions: Embedding dimension (384 for small, 1536 for large)
    """
    # Use text-embedding-3-small for 384 dimensions (default)
    # Use text-embedding-3-large for 1536 dimensions
    # Or use text-embedding-3-small with dimensions parameter for custom sizes
    # Option 1: Use large model (produces 1536 dimensions)
    response = openai.embeddings.create(
        input=text,
        model=model,
        dimensions=dimensions
    )

    embedding = response.data[0].embedding
    # Convert numpy types to native Python types for JSON serialization
    if isinstance(embedding, np.ndarray):
        return [float(x) for x in embedding]
    elif isinstance(embedding, list):
        return [float(x) if isinstance(x, (np.floating, np.float64, np.float32)) else x for x in embedding]

    return embedding


In [55]:
get_embedding(df.iloc[0]["description"])

[-0.004501929506659508,
 0.0068719275295734406,
 0.02274060808122158,
 0.013992579653859138,
 0.03462257608771324,
 -0.02357916720211506,
 0.014866672456264496,
 0.029591215774416924,
 0.05236024782061577,
 -0.0035105813294649124,
 0.02700447104871273,
 -0.004448631312698126,
 0.006424221675843,
 0.0057846419513225555,
 0.04187114164233208,
 0.011647454462945461,
 -0.040393002331256866,
 0.013850451447069645,
 -0.06225241348147392,
 0.02613748610019684,
 0.052502378821372986,
 0.01677119731903076,
 -0.02282588556408882,
 0.015889998525381088,
 -0.013239297084510326,
 -0.017979292199015617,
 -0.025853227823972702,
 -0.009764248505234718,
 0.003968946635723114,
 0.012464695610105991,
 0.012443375773727894,
 -0.046817224472761154,
 -0.0058699194341897964,
 -0.033314988017082214,
 0.02065131440758705,
 0.007245015352964401,
 -0.023920277133584023,
 0.02102084830403328,
 0.03826107084751129,
 0.014454498887062073,
 -0.006996289826929569,
 -0.0801037922501564,
 0.017581332474946976,
 0.04977

In [56]:
# only pick the description no na
df = df[df["description"].notna()]

df["embedding"] = df["description"].apply(lambda x: get_embedding(x))


In [57]:
print(len(df))

15


In [58]:
for embedding in df.embedding.tolist():
    print(f"embedding type: {type(embedding)}, value: {embedding}, length: {len(embedding)}")

("embedding type: <class 'list'>, value: [-0.004501929506659508, "
 '0.0068719275295734406, 0.02274060808122158, 0.013992579653859138, '
 '0.03462257608771324, -0.02357916720211506, 0.014866672456264496, '
 '0.029591215774416924, 0.05236024782061577, -0.0035105813294649124, '
 '0.02700447104871273, -0.004448631312698126, 0.006424221675843, '
 '0.0057846419513225555, 0.04187114164233208, 0.011647454462945461, '
 '-0.040393002331256866, 0.013850451447069645, -0.06225241348147392, '
 '0.02613748610019684, 0.052502378821372986, 0.01677119731903076, '
 '-0.02282588556408882, 0.015889998525381088, -0.013239297084510326, '
 '-0.017979292199015617, -0.025853227823972702, -0.009764248505234718, '
 '0.003968946635723114, 0.012464695610105991, 0.012443375773727894, '
 '-0.046817224472761154, -0.0058699194341897964, -0.033314988017082214, '
 '0.02065131440758705, 0.007245015352964401, -0.023920277133584023, '
 '0.02102084830403328, 0.03826107084751129, 0.014454498887062073, '
 '-0.0069962898269295

In [59]:
for topic in df.topics.tolist():
    print(f"type: {type(topic)}, value: {topic}, length: {len(topic)}")

"type: <class 'str'>, value: [], length: 2"
'type: <class \'str\'>, value: ["git","pre-commit-hook"], length: 25'
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"
("type: <class 'str'>, value: "
 '["ast","code-analysis","code-visualization","codevisualizer","diagrams","visualization","vscode","vscode-extension"], '
 'length: 116')
"type: <class 'str'>, value: [], length: 2"
("type: <class 'str'>, value: "
 '["aigc","comfyui","image-generation","tts","video-generation"], length: 62')
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"
"type: <class 'str'>, value: [], length: 2"


In [60]:
# Create metadatas as list of dictionaries

metadatas = []

for topic in df.topics.tolist():
    parsed = json.loads(topic)
    if isinstance(parsed, list):
        # ✅ Convert list to JSON string
        metadatas.append({"topics": json.dumps(parsed) if parsed else "[]"})
    # ... handles other cases too

print(len(metadatas))
print(metadatas)


15
[{'topics': '[]'},
 {'topics': '["git", "pre-commit-hook"]'},
 {'topics': '[]'},
 {'topics': '[]'},
 {'topics': '[]'},
 {'topics': '[]'},
 {'topics': '[]'},
 {'topics': '[]'},
 {'topics': '[]'},
 {'topics': '["ast", "code-analysis", "code-visualization", "codevisualizer", '
            '"diagrams", "visualization", "vscode", "vscode-extension"]'},
 {'topics': '[]'},
 {'topics': '["aigc", "comfyui", "image-generation", "tts", '
            '"video-generation"]'},
 {'topics': '[]'},
 {'topics': '[]'},
 {'topics': '[]'}]


In [61]:
print(df["description"].tolist())

['crypto-trading-open',
 'AI-powered git commit message rewriter using GPT',
 'AI writing agent powered by kimi-k2-thinking - autonomously creates novels '
 'and stories with deep reasoning',
 'voice activated sticker dreamer and printer. ',
 'Claude Code & Codex 多供应商代理与管理工具',
 'Venom C2 is a dependency‑free Python3 Command & Control framework for '
 'redteam persistence',
 'An enhanced Wan2.2 Image-to-Video node specifically designed to fix the '
 'slow-motion issue in 4-step LoRAs (like lightx2v).',
 '🔥windows key gen🔥 windows key generator windows 10 key generator windows 11 '
 'key generator windows 7 key generator 🔥windows key gen🔥 windows key '
 'generator windows 10 key generator windows 11 key generator windows 7 key '
 'generator 🔥windows key gen🔥 windows key generator windows 10 key generator '
 'windows 11 key generator windows 7 key e',
 'Offical code for "FastGS: Training 3D Gaussian Splatting in 100 Seconds"',
 'CodeVisualizer is a powerful VS Code extension that provides

In [62]:
# Add to ChromaDB
collection.add(
    documents=df['description'].tolist(),
    metadatas=metadatas,  # List of dictionaries
    ids=[str(identifier) for identifier in df['full_name'].tolist()],
    embeddings=df['embedding'].tolist()
)

print(f"\nSuccessfully added {len(df)} documents to ChromaDB collection")


'\nSuccessfully added 15 documents to ChromaDB collection'


In [ ]:
query_embedding = get_embedding("AI-powered software")

In [65]:
print(query_embedding)

[-0.02087286300957203,
 -0.013203768990933895,
 -0.012018729001283646,
 0.006864478345960379,
 0.021855907514691353,
 -0.043038494884967804,
 -0.03377363830804825,
 0.06646996736526489,
 -0.009386055171489716,
 0.0033985732588917017,
 0.018947172909975052,
 -0.04255370795726776,
 -0.03595518693327904,
 -0.030272383242845535,
 -0.009911242872476578,
 0.0010125021217390895,
 0.0021630346309393644,
 0.0053831785917282104,
 -0.013331699185073376,
 -0.02112872339785099,
 0.006005997769534588,
 0.03759808465838432,
 0.017183078452944756,
 0.008887799456715584,
 0.008113483898341656,
 0.00450113182887435,
 -0.01835465244948864,
 0.013883819803595543,
 0.014516739174723625,
 -0.02394319325685501,
 -0.027202052995562553,
 -0.01041623204946518,
 -0.012409253045916557,
 -0.03431229293346405,
 0.010072839446365833,
 -0.004117340315133333,
 -0.02363346703350544,
 0.01574217714369297,
 -0.013675091788172722,
 0.017236944288015366,
 -0.002942400285974145,
 -0.008618472144007683,
 0.017156146466732025

In [66]:
collection.query(
    query_embeddings=query_embedding,
    n_results=1
)

{'ids': [['xPOURY4/CodeCraft-Architect']],
 'embeddings': None,
 'documents': [['AI-powered software architect and full-stack engineer prompt that elevates web code development by enforcing production-grade architecture, consistent coding standards, and automated quality practices. Designed to boost developer productivity and code quality when used as the primary prompt.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None]],
 'distances': [[0.7352948188781738]]}